In [1]:
# Importing packages
import numpy as np
from sympy import solve, symbols, Eq
import random 

# Importing openfermion
import openfermion as of
from openfermionpyscf import generate_molecular_hamiltonian

/home/bowy/.local/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Setting h5py.default_file_mode is deprecated. 'r' (read-only) is the default from h5py 3.0.
  h5py.get_config().default_file_mode = 'r'


In [2]:
# Defining Pauli matrices and the identity matrix
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Defining the Hamiltonian
H = np.kron(Z, Z) + np.kron(X, X)

# Defining the measurement choices for Alice and Bob
A_0 = X
A_1 = Z
B_0 = 1/np.sqrt(2)*(X+Z)
B_1 = 1/np.sqrt(2)*(X-Z)

# Defining expansion coefficients C_ij
coeffs = [C_00, C_01, C_10, C_11] = symbols(('C_00', 'C_01', 'C_10', 'C_11'))

# Defining the Bell operator
Bell_operator = coeffs[0]*np.kron(A_0, B_0) + coeffs[1]*np.kron(A_0, B_1) + coeffs[2]*np.kron(A_1, B_0) + coeffs[3]*np.kron(A_1, B_1)

# Creating the sytsem of equations
Ops = [X, Z]
eqs = []
for i in range(len(Ops)):
    for j in range(len(Ops)):
        # Defining the projector operator
        Projector = np.kron(Ops[i],Ops[j])

        # calculating system of equations
        eqs.append( 
            Eq( 
                np.trace( np.matmul(Bell_operator, Projector) ), 
                np.trace( np.matmul(H, Projector) )
            ) 
        )

# Solving the system of equations
ans = solve(eqs, coeffs)

# Substituting the results in the Bell operator
Bell_operator = float(ans[C_00])*np.kron(A_0, B_0) + float(ans[C_01])*np.kron(A_0, B_1) + float(ans[C_10])*np.kron(A_1, B_0) + float(ans[C_11])*np.kron(A_1, B_1)

# Calculating the eigenvalues and the lowest 
eig_vals, eig_vecs = np.linalg.eigh(H)

# extracting the ground state energy and the respective eigenstate
E_G = eig_vals[0]
psi_G = eig_vecs[:,0]
psi_G_dagger = np.conjugate(np.transpose(psi_G))

# Adding the ground state energy to the Bell operator 
# Bell_operator += -E_G * np.kron(I,I)

# Calculating the quantum value
beta_Q = np.matmul(psi_G_dagger, np.matmul(Bell_operator, psi_G))

# Calculating the classical bound
m = [-1, 1]
I_Bell = []
for A_0 in m:
    for A_1 in m:
        for B_0 in m:
            for B_1 in m:
                I_Bell.append( 
                    float(ans[C_00])*A_0*B_0 + float(ans[C_10])*A_1*B_0 + float(ans[C_01])*A_0*B_1 + float(ans[C_11])*A_1*B_1
                 )

beta_C = -np.min(I_Bell)


In [23]:
# Set molecule parameters
geometry = [('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.401))]
basis = 'sto-3g'
multiplicity = 1
charge = 0

# Perform electronic structure calculations and obtain Hamiltonian as an InteractionOperator
hamiltonian = generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)

# Convert to a FermionOperator
hamiltonian_ferm_op = of.get_fermion_operator(hamiltonian)

# Obtaining the Jordan Wigner or Bravyi Kitaev Hamiltonian
qubit_hamiltonian = of.jordan_wigner(hamiltonian_ferm_op)

In [24]:
# Initializing the pauli matrices and strings
pauli_matrices = [I, X, Y, Z]
pauli_strings = ['I', 'X', 'Y', 'Z']

# Initializing the hamiltonian
N = hamiltonian_pauli.many_body_order()
H = np.zeros((2**N, 2**N))

# Calculating the Hamiltonian
for key in qubit_hamiltonian.terms.keys():
    # Initializing the indices
    idx = [0, 0, 0, 0]

    # Updating the possible indices
    for i in range(len(key)):
        for j in range(len(pauli_strings)):
            if key[i][1] == pauli_strings[j]:
                idx[key[i][0]] = j

    # Defining the Pauli chain
    p_chain = np.kron(np.kron(np.kron(pauli_matrices[idx[0]], pauli_matrices[idx[1]]), pauli_matrices[idx[2]]), pauli_matrices[idx[3]])

    # Adding terms to the Hamiltonian
    H += np.real( qubit_hamiltonian.terms[key] * np.array(p_chain) )


In [25]:
w, v = np.linalg.eigh(H)

In [26]:
w

array([-1.01528253, -0.87560215, -0.87560215, -0.87560215, -0.68263525,
       -0.68263525, -0.56407284, -0.56407284, -0.42942659, -0.36852269,
       -0.36852269, -0.2807903 , -0.2807903 , -0.26965961,  0.17141912,
        0.37771393])

In [27]:
qubit_hamiltonian

(-0.47400526839102486+0j) [] +
(-0.055771945840508375+0j) [X0 X1 Y2 Y3] +
(0.055771945840508375+0j) [X0 Y1 Y2 X3] +
(0.055771945840508375+0j) [Y0 X1 X2 Y3] +
(-0.055771945840508375+0j) [Y0 Y1 X2 X3] +
(0.10046123753968506+0j) [Z0] +
(0.14117274531209834+0j) [Z0 Z1] +
(0.08674372699665778+0j) [Z0 Z2] +
(0.14251567283716615+0j) [Z0 Z3] +
(0.10046123753968506+0j) [Z1] +
(0.14251567283716615+0j) [Z1 Z2] +
(0.08674372699665778+0j) [Z1 Z3] +
(-0.048887536852096236+0j) [Z2] +
(0.14888024846008352+0j) [Z2 Z3] +
(-0.048887536852096236+0j) [Z3]